# 1.2 Running Umbrella Sampling on NaCl

In [71]:
## import required packages
import os, math, sys
import numpy as np
import matplotlib.pyplot as plt
import simtk.openmm.app  as omm_app
import simtk.openmm as omm
import simtk.unit as unit
from tqdm import tqdm
import mdtraj
from FastMBAR import *
from tqdm.notebook import tqdm_notebook

In [72]:
psf_file = 'toppar/step3_input.psf'
pdb_file = 'toppar/step3_input.pdb'

In [73]:
## read the OpenMM system 
with open("output_files/NaCl_solvated_system", 'r') as file_handle:
    xml = file_handle.read()
system = omm.XmlSerializer.deserialize(xml)

## read psf and pdb file of butane
psf = omm_app.CharmmPsfFile(psf_file)
pdb = omm_app.PDBFile(pdb_file)

In [74]:
#### setup an OpenMM context

## platform
#platform = omm.Platform.getPlatformByName('CUDA') # If you have GPU try this 
platform = omm.Platform.getPlatformByName('OpenCL') # Or this one, if the preivous does not work
#platform = omm.Platform.getPlatformByName('CPU') # Try first using CUDA or OpenCL it is way faster

## intergrator
T = 298.15 * unit.kelvin  ## temperature
fricCoef = 10/unit.picoseconds ## friction coefficient 
stepsize = 2 * unit.femtoseconds ## integration step size
integrator = omm.LangevinIntegrator(T, fricCoef, stepsize)

## construct an OpenMM context
context = omm.Context(system, integrator, platform)

In [75]:
## set force constant K for the biasing potential. 
## the unit here is kJ*mol^{-1}*nm^{-2}, which is the default unit used in OpenMM
K = 12500
context.setParameter("k", K)

## M centers of harmonic biasing potentials
M = 15
umbrella_centers = np.linspace(0.2, 0.7, M, endpoint=False)
np.savetxt("output_files/bias.csv", umbrella_centers, delimiter = ",")

In [76]:
(umbrella_centers*10)

array([2.        , 2.33333333, 2.66666667, 3.        , 3.33333333,
       3.66666667, 4.        , 4.33333333, 4.66666667, 5.        ,
       5.33333333, 5.66666667, 6.        , 6.33333333, 6.66666667])

In [78]:
import time
## the main loop to run umbrella sampling window by window
st = time.time()

for window in range(M):
    s = time.time()
    print(f"Sampling at window: {window+1} out of {M}")
    
    ## set the center of the biasing potential
    context.setParameter("r0", umbrella_centers[window])

    ## minimize
    context.setPositions(pdb.positions)
    state = context.getState(getEnergy = True)
    energy = state.getPotentialEnergy()
    for i in tqdm_notebook(range(50), desc="Minimization Win {}".format(window+1)):
        omm.LocalEnergyMinimizer_minimize(context, 1, 20)
        state = context.getState(getEnergy = True)
        energy = state.getPotentialEnergy()
        
    print("Minimization in:", str(time.time() - s), "seconds")
    s = time.time()
    
    ## initial equilibrium
    integrator.step(500)
    
    print("Equilibration in:", str(time.time() - s), "seconds")
    s = time.time()

    ## sampling production. trajectories are saved in dcd files
    #file_handle = open(f"trajectories/traj_{window+1}_US_test.dcd", 'bw')
    file_handle = open(f"trajectories/long_traj/traj_win{window+1}_US.dcd", 'bw')
    dcd_file = omm_app.dcdfile.DCDFile(file_handle, psf.topology, dt = stepsize)
    for i in tqdm_notebook(range(5000), desc="Production"):
        integrator.step(100)
        state = context.getState(getPositions = True)
        positions = state.getPositions()
        dcd_file.writeModel(positions)
    file_handle.close()
    print("Production Run in:", str(time.time() - s), "seconds")

Sampling at window: 1 out of 15


Minimization Win 1:   0%|          | 0/50 [00:00<?, ?it/s]

Minimization in: 3.0257511138916016 seconds
Equilibration in: 0.01801609992980957 seconds


Production:   0%|          | 0/5000 [00:00<?, ?it/s]

Production Run in: 176.67200207710266 seconds
Sampling at window: 2 out of 15


...

Production Run in: 177.6975429058075 seconds
Sampling at window: 15 out of 15


Minimization Win 15:   0%|          | 0/50 [00:00<?, ?it/s]

Minimization in: 2.870609998703003 seconds
Equilibration in: 0.020018339157104492 seconds


Production:   0%|          | 0/5000 [00:00<?, ?it/s]

Production Run in: 173.38160920143127 seconds
